#### Setup

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import json
import sys
from datetime import datetime

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.metrics import r2_score

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


### Start Program

In [2]:
'''
Parameters 
----------
Set for each test. 


img_folder: Root folder of image collection

results_file: JSON file for output of results and metadata

description: String for labeling/notes

sample_size: Sample size to pull from each csv, 0-1

img_size: Native resolution is 1280x1280

'''

img_folder = '../data/output_images/'

results_file = '../data/results/'+'results3.json'

description = 'All ten image sets. Three quarters resolution.'

sample_size = 1

img_size = (1280,1280)

In [3]:
'''
Loads csv only, no images.
'''

# Name of folder
names = [
    'Australia',
    'China',
    'Germany',
    'NewarkLR',
    'Switzerland',
    'Amtrak',
    'BostonMTBA',
    'DenverRTD',
    'LosAngelesMR',
    'SeattleLLR',
    'Netherlands'
]

# Name of csv
abbr = [
    'AUS',
    'CHN',
    'GRM',
    'NEW',
    'SWZ',
    'AMT',
    'BOS',
    'DEN',
    'LAA',
    'SEA',
    'NET'
]
locations = dict(zip(names,abbr))

# Collect each csv into one df adding railway name
frames = []
for key,value in locations.items():
    try:
        filename = img_folder+key+'/'+value+'.csv'
        tmp = pd.read_csv(filename,header=0)
        tmp['Railway'] = key
        
        # Take sample from each folder 
        tmp = tmp.sample(frac=sample_size).reset_index(drop=True)
        frames.append(tmp)
    except Exception as e:
        print(e)

df = pd.concat(frames)

df = df.dropna()
df['Catenary'] = df['Catenary'].astype(int)

df.head()

File b'../data/output_images/China/CHN.csv' does not exist


,Name,Longitude,Latitude,Catenary,Railway
0,153.0331517_-27.4483646,153.033152,-27.448365,1,Australia
1,151.17792749999998_-33.5829912,151.177927,-33.582991,1,Australia
2,151.19101840000002_-33.9324425,151.191018,-33.932443,1,Australia
3,153.01603169999998_-27.4651858,153.016032,-27.465186,1,Australia
4,153.13842830000002_-30.2993999,153.138428,-30.299400,1,Australia


In [ ]:
df = df[df['Catenary']==0]

In [5]:
ones = df[df['Catenary']==1]
zeros = df[df['Catenary']==0]

In [ ]:
df.Catenary.value_counts()

In [4]:
'''
Open known non-catenary lines and add differntial to df
'''

zeros = df.Catenary.value_counts()[0]
ones = df.Catenary.value_counts()[1]

names = [
    'Amtrak_non_cat_1',
    'Amtrak_non_cat_2',
    'Amtrak_non_cat_3'
]

abbr = [
    'ANC',
    'ANC2',
    'ANC3'
]
locations = dict(zip(names,abbr))

diff = ones - zeros

if diff > 0:
    frames = []
    for key,value in locations.items():
        try:
            filename = img_folder+key+'/'+value+'.csv'
            tmp = pd.read_csv(filename,header=0)
            tmp['Railway'] = key
            frames.append(tmp)
        except Exception as e:
            print(e)

    try:
        duds = pd.concat(frames)
        duds = duds.dropna()
        duds['Catenary'] = duds['Catenary'].astype(int) 
        
        duds = duds.sample(n=diff).reset_index(drop=True)
        df = pd.concat([df,duds]).reset_index(drop=True)
    except Exception as e:
        print(e)
        duds = duds.sample(len(duds.index.tolist())).reset_index(drop=True)
        df = pd.concat([df,duds]).reset_index(drop=True)
        
df.shape

(640, 5)

In [ ]:
df.Catenary.value_counts()

In [6]:
'''
Load images into df
'''
rows = ones.index.tolist()

images = []
for row in rows:
    img_path = img_folder+df.iloc[row]['Railway']+'/'+df.iloc[row]['Name']+'.png'
    img = Image.open(img_path).convert('RGBA')
    img.thumbnail(img_size, Image.ANTIALIAS)
#     data = np.asarray(img)
#     data = data.flatten()
    # Append img instead of data if you want as image       
    images.append(img)
    
ones['Image'] = images

cols = ['Catenary','Image']
ones = ones[cols]

ones.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Catenary,Image
0,1,<PIL.Image.Image image mode=RGBA size=1280x128...
1,1,<PIL.Image.Image image mode=RGBA size=1280x128...
2,1,<PIL.Image.Image image mode=RGBA size=1280x128...
3,1,<PIL.Image.Image image mode=RGBA size=1280x128...
4,1,<PIL.Image.Image image mode=RGBA size=1280x128...


In [7]:
len(ones.index.tolist())

320

In [11]:
images = ones['Image'].tolist()

i = 0
for image in images[288:]:
    image.save('../data/output_images/validation/'+str(i)+'.png')
    i += 1

OSError: [Errno 28] No space left on device

In [ ]:
df['Image']

### Loading images into tf

In [ ]:
labels = np.asarray(df.Catenary.tolist())
features = np.asarray(df.Image.tolist())

In [ ]:
features = features.reshape(len(features),-1)

In [ ]:
features.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size = 0.20, random_state=42)

X_train, X_test = X_train / 255.0, X_test / 255.0

In [ ]:
len(X_test)

In [ ]:
with tf.device('/gpu:0'):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(input_shape=(28,28)),
        tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=5)

    model.evaluate(X_test, y_test)

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../data/output_images/train/',
        target_size=(66, 66),
        batch_size=32,
        class_mode='binary')

# validation_generator = test_datagen.flow_from_directory(
#         '../data/output_images/test',
#         target_size=(20, 20),
#         batch_size=32,
#         class_mode='binary')


# model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(32, activation=tf.nn.relu),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Flatten(input_shape=(28,28)),
#         tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)
#     ])

# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit_generator(
#         train_generator,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=800)

In [ ]:
with tf.device('/gpu:0'):
    rows = df.index.tolist()

    images = []
    for row in rows:
        img_path = img_folder+df.iloc[row]['Railway']+'/'+df.iloc[row]['Name']+'.png'
        img = Image.open(img_path).convert('RGBA')
        img.thumbnail(img_size, Image.ANTIALIAS)
        data = np.asarray(img)
    #     data = data.flatten()
        # Append img instead of data if you want as image       
        images.append(data)

    df['Image'] = images

    cols = ['Catenary','Image']
    df = df[cols]

    labels = np.asarray(df.Catenary.tolist())
    features = np.asarray(df.Image.tolist())
    
    features = features.reshape(len(features),-1)
    
    X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size = 0.20, random_state=42)

    X_train, X_test = X_train / 255.0, X_test / 255.0
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(input_shape=(28,28)),
        tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=5)

    model.evaluate(X_test, y_test)

### Basic Example

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
len(x_train)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test)

### Loading data example

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))